In [27]:
!git clone https://github.com/Gurupatil0003/Chest-X-Ray-Dataset


fatal: destination path 'Chest-X-Ray-Dataset' already exists and is not an empty directory.


In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [29]:
IMG_SIZE = 64
BATCH_SIZE = 32


In [30]:
train_path = "/content/Chest-X-Ray-Dataset/chest_xray/train"
test_path = "/content/Chest-X-Ray-Dataset/chest_xray/test"


In [31]:
train_data = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    train_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

test_data = ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    test_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary"
)


Found 693 images belonging to 2 classes.
Found 333 images belonging to 2 classes.


In [32]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [34]:
model.fit(
    train_data,
    validation_data=test_data,
    epochs=5
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 18s 732ms/step - accuracy: 0.9304 - loss: 0.3462 - val_accuracy: 0.5946 - val_loss: 0.8304
Epoch 2/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 17s 772ms/step - accuracy: 0.9310 - loss: 0.2656 - val_accuracy: 0.5946 - val_loss: 0.9518
Epoch 3/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 15s 693ms/step - accuracy: 0.9142 - loss: 0.2278 - val_accuracy: 0.5946 - val_loss: 1.1983
Epoch 4/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 15s 698ms/step - accuracy: 0.9007 - loss: 0.1705 - val_accuracy: 0.5946 - val_loss: 0.8714
Epoch 5/5
22/22 ━━━━━━━━━━━━━━━━━━━━ 15s 698ms/step - accuracy: 0.9143 - loss: 0.1184 - val_accuracy: 0.6877 - val_loss: 0.9525


In [35]:
model.save("chest_xray_model.h5")


In [36]:
!pip install gradio


In [37]:
import gradio as gr
import numpy as np
from PIL import Image

class_name = ["Normal", "Pneumonia"]

def predict(image):
    image = image.resize((IMG_SIZE, IMG_SIZE))
    image_array = np.array(image) / 255.0
    image_array = image_array.reshape(1, IMG_SIZE, IMG_SIZE, 3)
    prediction = model.predict(image_array)[0][0]
    label = class_name[1] if prediction >= 0.5 else class_name[0]
    confidence = prediction if prediction >= 0.5 else 1 - prediction
    return f"{label} (confidence: {confidence * 100:.2f}%)"


In [38]:
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type='pil'),
    outputs=gr.Textbox(label="Prediction")
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://733966edc0313faf7e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
